In [ ]:
from bs4 import BeautifulSoup
import requests
import simplejson as json
import ast
import csv
import pandas as pd
import tqdm

array_of_urls = ["/v-by-very-embroidered-lace-insert-skater-dress/1600177942.prd",
       "/v-fit-folding-x-frame-cycle-exercise-bike/1148569588.prd"]


In [ ]:
array_of_product_information = []
with open('product_urls.csv', 'r') as f:
    for line in f.readlines():
        url = line.strip().split(',')
        array_of_product_information.append((url[1]))

In [ ]:
array_of_urls = []
for x in array_of_product_information[1:]:
    array_of_urls.append(x)

In [ ]:
def request_web_data(url, brand_domain):
    #Function that handles the request to get the web page. 
    #Consider adding in a sleep at this point if crawl 
    #requests get a bit to much.
    
    r  = requests.get(brand_domain + url)
    web_page_data = r.text
    
    return web_page_data


def parse_web_data(web_page_data):
    #Parses and cleans the scraped data. If any more 
    #data is needed, then get it from the soup object
    
    soup = BeautifulSoup(web_page_data, "html.parser")
    description = soup.find(itemprop="description")
    
    try:
        h2 = description.h2.get_text()
        description.h2.string = ""
        header = 'yes'
    except:
        h2 = ''
        header = 'no'
   
    try:
        text = description.get_text(strip='\n')
        content = 'yes'
    except:        
        text = ''
        content = 'no'
    
    try:
        sku = soup.find('input', attrs={"class":"speedtrapProductID"})
        sku = sku['value']
        sku_found = 'yes'
    except:
        sku = ''
        sku_found = 'no'
    
    data_dictionary = {'sku':sku, 'sku_found': sku_found, 'h2': h2, 'header':header, 'text': text , 'content': content}
    
    return data_dictionary

def make_data_frame(array_of_urls):
    #Runs through an array of URL and outputs a dataframe
    #at the end. 
    
    result = []
    df = pd.DataFrame(columns=['sku', 'sku_found', 'h2', 'header', 'text', 'content'])
    for url in array_of_urls:
        data = request_web_data(url ,"http://www.very.co.uk")
        result.append(parse_web_data(data))
        
    df  = df.append(result, ignore_index=True)
        
    return df
        

In [ ]:
data_field = make_data_frame(array_of_urls)

In [ ]:
data_field['word count'] = data_field['text'].str.split().apply(len)

In [ ]:
data_field.to_csv('string_count.csv')


for url in array_of_urls:
    print(array_of_urls[url])

array_of_urls